In [1]:
import os
import time
import json
import pickle
import osmnx as ox
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [2]:


GRAPH_FILE = "new_delhi_road_network.graphml"   # change if needed
NODE_OUT_FILE = "node.extension"
ROAD_OUT_FILE = "road.extension"

# ---------- Load graph ----------
print("Loading graph...")
G = ox.load_graphml(GRAPH_FILE)
print("Graph loaded:", len(G.nodes), "nodes,", len(G.edges), "edges")


Loading graph...
Graph loaded: 10128 nodes, 24354 edges


In [ ]:
import os
import json
import time
import random
import osmnx as ox
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderUnavailable, GeocoderTimedOut

GRAPH_FILE = "delhi_10km_drive.graphml"
NODE_OUT_FILE = "node.json"

print("Loading graph...")
G = ox.load_graphml(GRAPH_FILE)
print("Graph loaded:", len(G.nodes), "nodes,", len(G.edges), "edges")

# Load existing node.json if present (resume)
if os.path.exists(NODE_OUT_FILE):
    with open(NODE_OUT_FILE, "r", encoding="utf-8") as f:
        try:
            node_to_address = json.load(f)
        except json.JSONDecodeError:
            node_to_address = {}
else:
    node_to_address = {}

print("Already saved nodes:", len(node_to_address))

# Geocoder with a larger timeout
geolocator = Nominatim(user_agent="delhi-router", timeout=10)

def reverse_geocode_node(lat, lon, max_tries=5):
    """
    Reverse geocode with retries + exponential backoff.
    Returns address string or None.
    """
    delay = 2.0
    for attempt in range(max_tries):
        try:
            loc = geolocator.reverse((lat, lon), language="en")
            if loc and loc.address:
                return loc.address
            return None
        except (GeocoderTimedOut, GeocoderUnavailable):
            # backoff and retry
            time.sleep(delay + random.uniform(0, 0.5))
            delay *= 2
        except Exception:
            # unknown error, don't crash whole run
            time.sleep(delay)
            delay *= 2
    return None


# Iterate nodes one by one, save after each new mapping
all_nodes = list(G.nodes())
total = len(all_nodes)

for i, node_id in enumerate(all_nodes, start=1):
    key = str(node_id)

    # skip if already saved
    if key in node_to_address:
        if i % 500 == 0:
            print(f"Progress {i}/{total} (skipping saved)")
        continue

    lat = G.nodes[node_id]["y"]
    lon = G.nodes[node_id]["x"]

    addr = reverse_geocode_node(lat, lon)

    if addr is None:
        addr = f"{lat}, {lon}"  # fallback

    node_to_address[key] = addr

    # save immediately (atomic-ish safe write)
    tmp_file = NODE_OUT_FILE + ".tmp"
    with open(tmp_file, "w", encoding="utf-8") as f:
        json.dump(node_to_address, f, ensure_ascii=False, indent=2)
    os.replace(tmp_file, NODE_OUT_FILE)

    # polite delay so Nominatim doesn't block you
    time.sleep(1.0 + random.uniform(0, 0.3))

    if i % 100 == 0:
        print(f"Saved {len(node_to_address)} nodes so far... ({i}/{total})")

print("Done. Total saved nodes:", len(node_to_address))
print("File:", NODE_OUT_FILE)


Loading graph...
Graph loaded: 59405 nodes, 158731 edges
Already saved nodes: 9217
Progress 500/59405 (skipping saved)
Saved 9498 nodes so far... (800/59405)
Saved 9597 nodes so far... (900/59405)
Saved 9685 nodes so far... (1000/59405)
Saved 9783 nodes so far... (1100/59405)
Saved 9878 nodes so far... (1300/59405)
Saved 10032 nodes so far... (1500/59405)
Saved 10132 nodes so far... (1600/59405)
Saved 10232 nodes so far... (1700/59405)
Saved 10332 nodes so far... (1800/59405)
Saved 10432 nodes so far... (1900/59405)
Saved 10532 nodes so far... (2000/59405)
Saved 10632 nodes so far... (2100/59405)
Saved 10724 nodes so far... (2200/59405)
Saved 10824 nodes so far... (2300/59405)
Saved 10924 nodes so far... (2400/59405)
Saved 11024 nodes so far... (2500/59405)
Saved 11124 nodes so far... (2600/59405)
Saved 11224 nodes so far... (2700/59405)
Saved 11324 nodes so far... (2800/59405)
Saved 11424 nodes so far... (2900/59405)
Saved 11524 nodes so far... (3000/59405)
Saved 11624 nodes so far...

In [ ]:
import json

with open("node.json", "r", encoding="utf-8") as f:
    node_map = json.load(f)

with open("road.json", "r", encoding="utf-8") as f:
    road_map = json.load(f)

# example:
print(node_map["58056528"])
print(road_map["Nelson Mandela Marg"])


Total unique nodes to reverse-geocode: 10128


KeyboardInterrupt: 